# Calculating all SPEI-indices at 0.1 degrees (2000-2023) for every gridcell

This python script uses the xclim package to calculate the SPEI-1 to 24 for every gridcell at 0.1 degrees. When calcualted, I slice 2000-2023 from the data to save some space and make it quicker to save.

In [1]:
import xclim
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pint
from xclim import indices
from xclim.core import units
from xclim.indices import standardized_precipitation_evapotranspiration_index
import pandas as pd
import spei as si  # si for standardized index

xr.set_options(keep_attrs=True)

## Definitions

You can't do it in one chunk anymore because the data is so big. Do it in multiple chunks now. Set the offset to the same value as in the 0.5 degrees data. 

In [2]:
def SPEI_calculation_parts(pe, spei_period, offset, cal_start, cal_end, dir):
    print("calculating spei")
    
    lon_chunks = np.array_split(pe.lon.values, 20)  # Split longitude into 20 chunks
    for i, lon_chunk in enumerate(lon_chunks):
        pe_chunk = pe.sel(lon=lon_chunk)
        SPEI_chunk = standardized_precipitation_evapotranspiration_index(pe_chunk, window=spei_period, dist="fisk", freq="MS", offset=offset, cal_start=cal_start, cal_end=cal_end)

        del SPEI_chunk.attrs['freq']
        del SPEI_chunk.attrs['time_indexer']
        del SPEI_chunk.attrs['units']
        del SPEI_chunk.attrs['offset']
        SPEI_2000_2023 = SPEI_chunk.sel(time = slice("2000-01-01","2023-12-31"))
        print("saving part", i+1)
        SPEI_2000_2023.to_netcdf(path="/scratch/ruiij001/Data/SPEI/0_1_degrees_apr_2024/" + dir + f"_part_{i+1}.nc")
    
    print("done")

## Load the data

Do calculate the SPEI at 0.1 degrees, we first need to interpolate all the necesarry data: the land/sea mask, the precipitation and the pet. We chose to do this interpolation bilinearly. 

1. ncatted -O -a units,lon,c,c,"degrees_east" -a units,lat,c,c,"degrees_north" infile.nc outfile.nc #to gives them the right units

2. cdo remapbil,/scratch/ruiij001/Data/gridfile0_1.txt <infile.nc> <outfile.nc> #regrid to 0.1 degrees CDO command to regrid

Load the landmask first. The data is also over the ocean, but we don't need that.

In [3]:
landmask = xr.open_dataarray("/scratch/ruiij001/Data/Masks/land_sea/land-sea-mask_0_1_degrees.nc").mean("time")

Load the total precipitation data. Note that this data is in m, and we want it in mm. So we multiply by 1000. If necesarry, resample the data to monthly. Then use the land/sea mask to filter out only the land data.

In [4]:
total_prec_mm = (xr.open_dataset("/scratch/ruiij001/Data/ERA5/total_precip/new/era5_total_precipitation_1950-2023_monthly_0_1_bil_regridded.nc").tp*1000).where(landmask>=0.5)#.resample(time="MS").mean()
total_prec_mm

<xarray.DataArray 'tp' (time: 888, lat: 1800, lon: 3600)>
array([[[0.14808029, 0.14808029, 0.14808029, ..., 0.14808029,
         0.14808029, 0.14808029],
        [0.14808029, 0.14808029, 0.14808029, ..., 0.14808029,
         0.14808029, 0.14808029],
        [0.14808029, 0.14808029, 0.14808029, ..., 0.14808029,
         0.14808029, 0.14808029],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[0.0571534 , 0.0571534 , 0.0571534 , ..., 0.0571534 ,
         0.0571534 , 0.0571534 ],
        [0.05455315, 0.05455315, 0.05455315, ..., 0.05455315,
         0.05455315, 0.05455315],
        [0.05455315, 0.05455315, 0.05455315, ..., 0.05455315,
         0.05455315, 0.05455315],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[0.2156198 , 0.2156198 , 0.2156198 , ..., 0.2156198 ,
         0.2156198 , 0.2156198 ],
        [0.22860616, 0.22860616, 0.22860616, ..., 0.22860616,
         0.22860616, 0.22860616],
        [0.23899972, 0.23899972, 0.23899972, ..., 0.23899972,
         0.23899972, 0.23899972],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2023-12-01
  * lon      (lon) float64 -180.0 -179.9 -179.8 -179.7 ... 179.7 179.8 179.9
  * lat      (lat) float64 -90.0 -89.9 -89.8 -89.7 -89.6 ... 89.6 89.7 89.8 89.9
Attributes:
    long_name:  Total precipitation
    units:      m

Load the regridded PET data file. Note that this is monthly data now, so we don't have to resample anymore. Also use the land/sea mask.

In [5]:
pet = xr.open_dataset("/scratch/ruiij001/Data/PET/new/pm_fao56_1950_2023_monthly_0_1_degrees_bil_remapped.nc").PM_FAO_56.where(landmask>=0.5)
pet

<xarray.DataArray 'PM_FAO_56' (time: 888, lat: 1800, lon: 3600)>
array([[[0.06321488, 0.06321488, 0.06321488, ..., 0.06321488,
         0.06321488, 0.06321488],
        [0.06328619, 0.06329097, 0.06329574, ..., 0.06327122,
         0.06327621, 0.0632812 ],
        [0.06335751, 0.06336707, 0.06337662, ..., 0.06332756,
         0.06333754, 0.06334753],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[0.0246702 , 0.0246702 , 0.0246702 , ..., 0.0246702 ,
         0.0246702 , 0.0246702 ],
        [0.0251888 , 0.02519006, 0.02519133, ..., 0.02518564,
         0.0251867 , 0.02518775],
        [0.02570739, 0.02570992, 0.02571245, ..., 0.02570108,
         0.02570319, 0.02570529],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[0.09756462, 0.09756462, 0.09756462, ..., 0.09756462,
         0.09756462, 0.09756462],
        [0.09749238, 0.09749614, 0.0974999 , ..., 0.09748069,
         0.09748459, 0.09748849],
        [0.09742015, 0.09742767, 0.09743519, ..., 0.09739676,
         0.09740456, 0.09741236],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2023-12-01
  * lon      (lon) float64 -180.0 -179.9 -179.8 -179.7 ... 179.7 179.8 179.9
  * lat      (lat) float64 -90.0 -89.9 -89.8 -89.7 -89.6 ... 89.6 89.7 89.8 89.9

Make sure the lon and lat values are the same for both datasets.

In [6]:
# Check if the longitude and latitude values are the same
lon_same = (pet['lon'] == total_prec_mm['lon']).all()
lat_same = (pet['lat'] == total_prec_mm['lat']).all()

if lon_same and lat_same:
    print("Longitude and Latitude values are the same for both datasets.")
else:
    print("Longitude and/or Latitude values are different between the datasets.")

Longitude and Latitude values are the same for both datasets.


If not: uncomment these lines

In [7]:
# pet['lon'] = total_prec_mm['lon']
# pet['lat'] = total_prec_mm['lat']

Calculate the water balance (P - PET), which is the input of the SPEI calculation. Make sure the units of the xarray are set to mm/d, because the xclim package wants this. If necesarry, select only the overlapping time periods ot filter out the bad data.

In [8]:
# pe = total_prec_mm.sel(time = slice("1955-01-01","2023-12-31")).assign_attrs(units='mm/d') - pet.sel(time = slice("1955-01-01","2023-12-31")).assign_attrs(units='mm/d')
pe = total_prec_mm.assign_attrs(units='mm/d') - pet.assign_attrs(units='mm/d')
pe

<xarray.DataArray (time: 888, lat: 1800, lon: 3600)>
array([[[0.08486541, 0.08486541, 0.08486541, ..., 0.08486541,
         0.08486541, 0.08486541],
        [0.0847941 , 0.08478932, 0.08478455, ..., 0.08480907,
         0.08480408, 0.08479909],
        [0.08472278, 0.08471322, 0.08470367, ..., 0.08475273,
         0.08474275, 0.08473276],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[0.0324832 , 0.0324832 , 0.0324832 , ..., 0.0324832 ,
         0.0324832 , 0.0324832 ],
        [0.02936435, 0.02936309, 0.02936182, ..., 0.02936751,
         0.02936646, 0.02936541],
        [0.02884576, 0.02884323, 0.0288407 , ..., 0.02885207,
         0.02884996, 0.02884786],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[0.11805519, 0.11805519, 0.11805519, ..., 0.11805519,
         0.11805519, 0.11805519],
        [0.13111378, 0.13111001, 0.13110626, ..., 0.13112548,
         0.13112158, 0.13111767],
        [0.14157957, 0.14157206, 0.14156453, ..., 0.14160296,
         0.14159516, 0.14158738],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2023-12-01
  * lon      (lon) float64 -180.0 -179.9 -179.8 -179.7 ... 179.7 179.8 179.9
  * lat      (lat) float64 -90.0 -89.9 -89.8 -89.7 -89.6 ... 89.6 89.7 89.8 89.9
Attributes:
    long_name:  Total precipitation
    units:      mm/d

## Calculate SPEI

Use the same offset as in the 0.5 degrees data. Then specify the start and the end of the calibration period. Note that the mean is now only 0 between the start and the end date. 

In [9]:
# Initial array
SPEIS = np.array([])

# Extend the array up to SPEI24
for i in range(4, 12):
    SPEIS = np.append(SPEIS, f"SPEI{i:02d}")

# Print the extended array
print(SPEIS)

SPEI_periods = np.arange(4,12)

['SPEI04' 'SPEI05' 'SPEI06' 'SPEI07' 'SPEI08' 'SPEI09' 'SPEI10' 'SPEI11']


In [10]:
offset = '20 mm/d'
cal_start = "1950-01-01"
cal_end = "2020-12-31"
for name, period in zip(SPEIS, SPEI_periods):
    print(period)
    dir = name+"_monthly_2000_2023_0_1_degree"
    SPEI_calculation_parts(pe = pe, spei_period = period, offset = offset, cal_start = cal_start, cal_end = cal_end, dir = dir)

4
calculating spei
saving part 1
saving part 2
saving part 3
saving part 4
saving part 5
saving part 6
saving part 7
saving part 8
saving part 9
saving part 10
saving part 11
saving part 12
saving part 13
saving part 14
saving part 15
saving part 16
saving part 17
saving part 18
saving part 19
saving part 20
done
5
calculating spei
saving part 1
saving part 2
saving part 3
saving part 4
saving part 5
saving part 6
saving part 7
saving part 8
saving part 9
saving part 10
saving part 11
saving part 12
saving part 13
saving part 14
saving part 15
saving part 16
saving part 17
saving part 18
saving part 19
saving part 20
done
6
calculating spei
saving part 1
saving part 2
saving part 3
saving part 4
saving part 5
saving part 6
saving part 7
saving part 8
saving part 9
saving part 10
saving part 11
saving part 12
saving part 13
saving part 14
saving part 15
saving part 16
saving part 17
saving part 18
saving part 19
saving part 20
done
7
calculating spei
saving part 1


KeyboardInterrupt: 

## Merge all the data together

In [ ]:
parts = xr.open_mfdataset("/scratch/ruiij001/Data/SPEI/0_1_degrees_apr_2024/SPEI01_monthly_2000_2023_0_1_degree.nc_part_*.nc").__xarray_dataarray_variable__
parts 

## Load the SPEI-12 data and make some test plots

Load the self made SPEI-12.

In [ ]:
SPEI_12 = xr.open_dataset("/scratch/ruiij001/Data/SPEI/0_5_degrees_apr_2024/" + dir).__xarray_dataarray_variable__

Look if the mean of the SPEI for the calibration period is equal to 0 and the standard deviation is equal to 1.

In [ ]:
SPEI_12.sel(time = slice(cal_start, cal_end)).mean("time").plot()

In [ ]:
SPEI_12.sel(time = slice(cal_start, cal_end)).std("time").plot()

Also load the SPEI-12 from the global SPEI database to see if they are the same. Keep in mind that both have a spatial resolution of 0.5 degrees, but that our spei is at .0 and .5, but the SPEI database grid points are at .25 and .75, so they won't line up perfectly. 

In [ ]:
SPEI_database =  xr.open_dataset("/scratch/ruiij001/Data/SPEI/spei12.nc").spei
SPEI_database

Make a timeseries for a certain longitude and latitude point.

In [ ]:
def plot_spei_timeseries(SPEI_12, SPEI_database, lon, lat, start, end):
    fig = plt.figure(figsize = (20,9))
    fig.tight_layout()
    fig.suptitle("SPEI-12")

    ax2 = fig.add_subplot(111)
    SPEI_12.sel(lon = lon, lat = lat, method = "nearest").sel(time = slice(start,end)).plot(ax=ax2, label = "SPEI own (calibration 1955-2020)")
    SPEI_database.sel(time = slice(start,end)).sel(lon = lon, lat = lat, method = "nearest").plot(ax = ax2, label = "SPEI database")

    ax2.set_ylim(-3,3)
    ax2.axhline(y=0, color="black", linestyle="--")
    ax2.axhline(y=1, color="lightblue", linestyle="--")
    ax2.axhline(y=-1, color="lightcoral", linestyle="--")
    ax2.set_ylabel("SPEI-12")
    ax2.legend()

You can take any point you want

In [ ]:
# point in middle of California
plot_spei_timeseries(SPEI_12 = SPEI_12, SPEI_database = SPEI_database, lon = -118, lat = 35, start = "1955" , end = "2018")

In [ ]:
# point in middle of Brazil
plot_spei_timeseries(SPEI_12 = SPEI_12, SPEI_database = SPEI_database, lon = -67, lat = -5, start = "1955" , end = "2018")

In [ ]:
# point in India
plot_spei_timeseries(SPEI_12 = SPEI_12, SPEI_database = SPEI_database, lon = 81, lat = 25, start = "1955" , end = "2018")

In [ ]:
# point in Australia
plot_spei_timeseries(SPEI_12 = SPEI_12, SPEI_database = SPEI_database, lon = 145, lat = -37, start = "1955" , end = "2018")